In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def prepare_data():
    """ 
    準備資料
    Returns:
        X_train(ndarray): 訓練資料 (50000.32.32.3)
        X_test(ndarray): 測試資料 (10000.32.32.3)
        y_train(ndarray): 將訓練資料的正確答案用One-hot encoding後表示 (50000,10)
        y_train(ndarray): 將測試資料的正確答案用One-hot encoding後表示 (10000,10)
        y_test_label(ndarray): 測試資料的正確答案 (10000)
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # 對訓練資料與測試資料的圖像執行正規化
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    # 將訓練資料與測試資料的的正確答案用One-hot encoding後表示
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    return x_train, x_test, y_train, y_test


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    """ 
    建立模型
    """
    # Sequential物件
    model = Sequential()
    # 卷積層 1
    model.add(Conv2D(filters=64, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu', 
                     input_shape=(32,32,3)))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 2
    model.add(Conv2D(filters=128, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu'))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 3
    model.add(Conv2D(filters=256, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu')) 
    #2 × 2 池化層 
    model.add(MaxPooling2D(pool_size=2)) 
    # 扁平層 
    model.add(Flatten()) 
    # 丟棄法
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 輸出層
    model.add(Dense(10, activation='softmax'))

    # 優化器為 Adam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])
    return model


In [ ]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras.callbacks import Callback

class LRHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.lr = []
    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.lr.append(step_decay(len(self.acc)))

def step_decay(epoch): 
    """
    運用步驟衰減讓學習率降低的函數  
    Returns: 學習率 (float) 
    """ 
    initial_lrate = 0.001   # 初始學習率 
    drop = 0.5              # 衰減率 
    epochs_drop = 10.0      # 每10個訓練週期後要進行學習率衰減
    lrate = initial_lrate * math.pow(drop,
                                     math.floor((epoch)/epochs_drop))
    return lrate

def train(x_train, x_test, y_train, y_test):

    model = make_convlayer()
    lr_history = LRHistory()
    lrate = LearningRateScheduler(step_decay)
    callbacks_list = [lr_history, lrate]

    # 資料擴增
    datagen = ImageDataGenerator(width_shift_range=0.1,  # 以圖像寬度的0.1比例隨機橫向移動
                                 height_shift_range=0.1, # 以圖像高度的0.1比例隨機上下移動
                                 rotation_range=10,      # 在10度的範圍內隨機旋轉
                                 zoom_range=0.1,         # 以原始尺寸的0.1比例隨機放大
                                 horizontal_flip=True)   # 左右翻轉

    # 批次大小
    batch_size = 128
    # 訓練週期
    epochs = 100

    # 訓練
    history = model.fit(datagen.flow(x_train, # 看批次大小多少，就建立多少的擴增資料
                                     y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,# 每次訓練當中的步驟次數
                        epochs=epochs,                                 # 訓練週期 
                        verbose=1,                                     # 輸出訓練進度的情況
                        validation_data=(x_test, y_test),              # 驗證資料
                        callbacks=callbacks_list
                       )

    return history, lr_history


In [ ]:
x_train, x_test, y_train, y_test = prepare_data()

In [ ]:
%%time
history, lr_history = train(x_train, x_test, y_train, y_test)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x_range = list(range(1, len(lr_history.lr)+1))

# 設定繪圖尺寸
plt.figure(figsize=(15, 10))     # 將圖形縮小、讓圖形之間保有空間 
plt.subplots_adjust(wspace=0.2)  # 在2 × 1 格線的上方繪圖 
plt.subplot(2, 1, 1)             # 繪製訓練資料與驗證資料的準確度 
plt.plot(x_range, history.history['accuracy'], label='train', linestyle='--')
plt.plot(x_range, history.history['val_accuracy'], label='Val Acc')
plt.legend()                     # 顯示圖例

plt.grid()                       # 顯示格線
plt.xlabel('Epoch')              # x 軸標籤
plt.ylabel('Acc')                # y 軸標籤

# 在2 × 1 格線的下方繪圖
plt.subplot(2, 1, 2)             # 繪製學習率 
plt.plot(x_range, lr_history.lr, label='Learning Rate')
plt.legend()                     # 顯示圖例
plt.grid()                       # 顯示格線
plt.xlabel('Epoch')              # x 軸標籤
plt.ylabel('Learning Rate')      # y 軸標籤
plt.show()
